In [382]:
import csv
import re
import pymysql
import pandas as pd
from scipy import stats
from  scipy.stats import chi2_contingency
import numpy as np
from functools import reduce
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_maqy',
                     password='6WVTr1$My4r99@&w',
                     database='asgard_dataplatform',
                     port=3001,
                     charset='utf8')

In [383]:
import os
path = 'C:\\Users\\86187\\Desktop\\13.tag正确率判断\\tag训练库整理\\tag训练库'
filelist=os.listdir(path)
filelist

['emr_214_test_out-已核对20200619.csv',
 'emr_214_valid_out-已核对20200619.csv',
 'emr_238_test_out-已核对.csv',
 'emr_238_valid_out-已核对.csv',
 'emr_242_test_out-已核对.csv',
 'emr_242_valid_out-已核对.csv',
 'emr_267_test_out-已核对.csv',
 'emr_267_valid_out-已核对.csv',
 'emr_308_test_out-已核对.csv',
 'emr_308_valid_out-已核对.csv']

In [384]:
#转换成excel
excelpath='C:\\Users\\86187\\Desktop\\13.tag正确率判断\\tag训练库整理\\tag训练库excel'
for name in filelist:
    filepath=path+'\\'+name
    df=pd.read_csv(filepath,encoding="gbk",dtype='str')
    filepath=excelpath+'\\'+name.replace('.csv','.xlsx')
    df.to_excel(excel_writer=filepath,index=False)

In [385]:
filelist=os.listdir(excelpath)
trainmerge=pd.DataFrame(columns=['line_number','tagname','tex'])
outputpath='C:\\Users\\86187\\Desktop\\13.tag正确率判断\\tag训练库整理\\tag训练库转换后'
for name in filelist:
    filepath=excelpath+'\\'+name
    df=pd.read_excel(filepath,dtype='str')
    #将列名标准化一下
    if 'model_tag错误' in df.columns.tolist():
        df.rename(columns={'model_tag错误':'model_tags错误'},inplace = True)
    if 'models_tags错误' in df.columns.tolist():
        df.rename(columns={'models_tags错误':'model_tags错误'},inplace = True)
    if 'tags_错误' in df.columns.tolist():
        df.rename(columns={'tags_错误':'tags错误'},inplace = True)
    if '正确标签' in df.columns.tolist():
        df.rename(columns={'正确标签':'正确的'},inplace = True)
    df['tag']=1
    #得到正确的tag表格
    for i in range(len(df)):
        if (df.iloc[i]['model_tags错误']=='TRUE') & (pd.isna(df.iloc[i]['tags错误'])==True):
            df.iloc[i,df.columns.tolist().index('tag')]=df.iloc[i]['tags']
        elif (pd.isna(df.iloc[i]['model_tags错误'])==True) & (df.iloc[i]['tags错误']=='TRUE'):
            df.iloc[i,df.columns.tolist().index('tag')]=df.iloc[i]['model_tags']
        elif (df.iloc[i]['model_tags错误']=='TRUE') & (df.iloc[i]['tags错误']=='TRUE'):
            df.iloc[i,df.columns.tolist().index('tag')]=df.iloc[i]['正确的']
    dfnew=df[['line_number','text','tag']]
    filepath=outputpath+'\\'+name.replace('.csv','.xlsx')
    dfnew.to_excel(excel_writer=filepath,index=False)
    
    #求tag数目总数目，然后计算需要多少行来匹配
    lenth=0
    for line in range(len(dfnew)):
        alltag=df.iloc[line]['tag'].split(',')
        lenth=len(alltag)+lenth
    train=pd.DataFrame(np.random.rand(lenth*3).reshape(lenth,3),columns=['line_number','tagname','tex'])
    #对于每一行，提取出tag和文本，放到新表里
    l=0
    for a in range(len(dfnew)):
        alltag=df.iloc[a]['tag'].split(',')    
        for b in range(len(alltag)):
            train.iloc[l+b,0]=dfnew.iloc[a]['line_number']
            train.iloc[l+b,1]=alltag[b]
            train.iloc[l+b,2]=dfnew.iloc[a]['text']      
        l=l+len(alltag)
    trainpath=outputpath+'\\'+'train'+name.replace('.csv','.xlsx')
    train.to_excel(excel_writer=trainpath,index=False)
    trainmerge = pd.concat([trainmerge,train],axis = 0)
trainmerge.to_excel(excel_writer='C:\\Users\\86187\\Desktop\\13.tag正确率判断\\tag训练库整理\\trainmerge.xlsx',index=False)  

In [402]:
trainmerge.to_excel(excel_writer='C:\\Users\\86187\\Desktop\\13.tag正确率判断\\tag训练库整理\\trainmerge.xlsx',index=False)  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34675 entries, 0 to 3548
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   line_number  34675 non-null  object
 1   tex          34675 non-null  object
dtypes: object(2)
memory usage: 812.7+ KB
